<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verificaiton/fitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import pickle

In [ ]:
def fit_naive(Model, Batcher, test_X, Loss, initial_epoch, max_epoch, optimizer, callbacks):
  Model.compile(optimizer=optimizer, loss=Loss, metrics=['accuracy'])
  Model.fit(x=Batcher.X, y=Batcher.Y, batch_size=100, epochs = max_epoch, initial_epoch=initial_epoch, shuffle=True, validation_data=(test_X,test_X), validation_batch_size=test_X.shape[0], callbacks = callbacks)

In [ ]:
def fit_simMat(Model, Batcher, test_X, Loss, initial_epoch, max_epoch, optimizer, callbacks):
  Model.compile(optimizer=optimizer, loss=Loss)  
  Model.fit(x=Batcher.train_generator(), steps_per_epoch=10, epochs = max_epoch, initial_epoch=initial_epoch, shuffle=False, validation_data=(test_X,test_X), validation_batch_size=test_X.shape[0], callbacks = callbacks)

In [ ]:
def fit(Model, Batcher, test_X, Loss, initial_epoch, max_epoch, optimizer, callbacks):
    print('==================================================')
    print('start training...')
    if Batcher.name == 'naive_batcher': fit_naive(Model, Batcher, test_X, Loss, initial_epoch, max_epoch, optimizer, callbacks)
    elif Batcher.name == 'simMat_batcher': fit_simMat(Model, Batcher, test_X, Loss, initial_epoch, max_epoch, optimizer, callbacks)